# Agent with MCP Tool
https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-projects/samples/agents/tools/sample_agent_mcp.py

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool
from openai.types.responses.response_input_param import McpApprovalResponse, ResponseInputParam


load_dotenv()

endpoint = os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"]

In [ ]:
with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=endpoint, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
):
    # [START tool_declaration]
    mcp_tool = MCPTool(
        server_label="api-specs",
        server_url="https://gitmcp.io/Azure/azure-rest-api-specs",
        require_approval="always",
    )
    # [END tool_declaration]

    agent = project_client.agents.create_version(
        agent_name="MyAgent",
        definition=PromptAgentDefinition(
            model=os.environ["AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME"],
            instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
            tools=[mcp_tool],
        ),
    )
    print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

    # Create a conversation thread to maintain context across multiple interactions
    conversation = openai_client.conversations.create()
    print(f"Created conversation (id: {conversation.id})")

    # Send initial request that will trigger the MCP tool
    response = openai_client.responses.create(
        conversation=conversation.id,
        input="Please summarize the Azure REST API specifications Readme",
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )

    # Process any MCP approval requests that were generated
    input_list: ResponseInputParam = []
    for item in response.output:
        if item.type == "mcp_approval_request":
            if item.server_label == "api-specs" and item.id:
                # Automatically approve the MCP request to allow the agent to proceed
                # In production, you might want to implement more sophisticated approval logic
                input_list.append(
                    McpApprovalResponse(
                        type="mcp_approval_response",
                        approve=True,
                        approval_request_id=item.id,
                    )
                )

    print("Final input:")
    print(input_list)

    # Send the approval response back to continue the agent's work
    # This allows the MCP tool to access the GitHub repository and complete the original request
    response = openai_client.responses.create(
        input=input_list,
        previous_response_id=response.id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )

    # Print result (should contain "Azure")
    print(f"==> Result: {response.output_text}")

    # Clean up resources by deleting the agent version
    # This prevents accumulation of unused agent versions in your project
    project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
    print("Agent deleted")

Agent created (id: MyAgent:5, name: MyAgent, version: 5)
Created conversation (id: conv_0c2ffe8f0912e82400vaDFIZuTisvcou2HYK0tD3AHKrxzuAwY)
==> Result: Here are the *live* London Underground updates from Transport for London right now (showing **Tube lines only**):

- **Bakerloo** – Good service  
- **Central** – Good service  
- **Circle** – Good service  
- **District** – Minor delays between *Earl’s Court and Ealing Broadway/Richmond* and *Barking to Upminster eastbound only* due to an earlier points failure. Good service on the rest of the line.   
- **Hammersmith & City** – Good service  
- **Jubilee** – Good service  
- **Metropolitan** – Minor delays between *Moor Park and Watford* due to train cancellations. Good service on the rest of the line.   
- **Northern** – Good service  
- **Piccadilly** – Minor delays between *Acton Town and Uxbridge* due to a points failure at Acton Town. Good service on the rest of the line.   
- **Victoria** – Good service  
- **Waterloo & City** –